## Deep Learning Assignment 1
___________
### Group Members
#### Danqing Yin - 430563708
#### Prerona Majumder - 470531989
#### Ryan Clark - 470295634

___________________

The section marked 'Add Inputs' is used to set file locations for testing run by assignment marker

Please complete teh file locations, 'final_test' locations are made for the unseen data

analysis can be set to True if the marker would like to see the built in training analysis

In [5]:
#Imports libraries needed
#      -- NOTE: Sklearn is only used outside of the Neural network for measurement

import matplotlib
import numpy as np
import h5py
from scipy import stats
from sklearn import metrics
import matplotlib.pyplot as pl
import time

###Add Inputs Here

In [6]:
#Add the input file locations here

training_data_file_location = 'Assignment1_Dataset/train_128.h5'
training_label_file_location = 'Assignment1_Dataset/train_label.h5'
final_test_file_location = 'Assignment1_Dataset/train_128.h5'
final_test_file_location = 'Assignment1_Dataset/train_label.h5'

#Set analysis = True to see training analysis
analysis = False


with h5py.File(training_data_file_location,'r') as H: final_test_data = np.copy(H['data'])
with h5py.File(training_label_file_location,'r') as H: final_test_label = np.copy(H['label'])



OSError: Unable to open file (unable to open file: name = '/input/train_128.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [3]:
class Softmax:
    
    @staticmethod
    def activation(z):
        """ Compute the Softmax function.
        """
        exps = np.exp(z - np.max(z))
        return exps / np.sum(exps)
        
class Tanh:        
    def activation(z):
        """ Compute the tanh function or its derivative.
        """
        return np.tanh(z)
    
    def prime(z):
        return 1 - np.square(Tanh.activation(z))
    

class Relu:
    @staticmethod
    def activation(z):
        """ Compute the Relu function or its derivative.
        """
        z[z < 0] = 0
        return z

    @staticmethod
    def prime(z):
        z[z < 0] = 0
        z[z > 0] = 1
        return z


class Sigmoid:
    @staticmethod
    def activation(z):
        """ Compute the Sigmoid function or its derivative.
        """
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def prime(z):
        return Sigmoid.activation(z) * (1 - Sigmoid.activation(z))
    
    
class CrossEntropy:
    def __init__(self, activation_fn=Softmax):
        self.m = None
        """
        :param activation_fn: Class object of the activation function.
        """
        if activation_fn:
            self.activation_fn = activation_fn
            print(activation_fn)
            
        else:
            self.activation_fn = NoActivation
            
    def activation(self, z):
        return self.activation_fn.activation(z)

    @staticmethod
    def loss(y_onehot, x):
        """ Compute the Cross Entropy loss function or its derivative.
        """
        indices = np.argmax(y_onehot, axis = 1).astype(int)
        #x = Softmax.activation(x)
        predicted_probability = x[np.arange(len(x)), indices]
        log_preds = np.log(predicted_probability)
        log_preds[np.isnan(log_preds)] = 0 
        loss = -1.0 * np.sum(log_preds) / len(log_preds)
        return loss
    
    def delta(self, y, a):
        """
        Cp_a, dC/da: the derivative of C w.r.t a
        ''a'' is the output of neurons
        ''y'' is the expected output of neurons
        """
        return (a - y) # delta
        
class Momentum():       
    """ Compute the Momentum
        """
    
    def __init__(self, weights, velocity, eta, dw):
        self.w = weights
        self.v = velocity
        self.eta = eta
        self.dw = dw
        
    def update(self):
        
        gamma = 0.1
        
        self.v = gamma*self.v + self.eta*self.dw
        self.w -= self.v
        
        return self.w, self.v        
        
class NoActivation:
    """
    This is a plugin function for no activation.
    f(x) = x * 1
    """
    @staticmethod
    def activation(z):
        """
        :param z: (array) w(x) + b
        :return: z (array)
        """
        return z

    @staticmethod
    def prime(z):
        """
        The prime of z * 1 = 1
        :param z: (array)
        :return: z': (array)
        """
        return np.ones_like(z)


class Network:
    def __init__(self, dimensions, activations):
        """
        Creates the layers a actiations of the network
        """
        self.n_layers = len(dimensions)
        self.loss = None
        self.learning_rate = None

  
        # Weights and biases are initiated by index. For a one hidden layer net you will have a w[1] and w[2]
        self.w = {}
        self.b = {}
        # velocity
        self.vel = {}

        # Activations are also initiated by index. For the example we will have activations[2] and activations[3]
        self.activations = {}
        for i in range(len(dimensions) - 1):
            self.w[i + 1] = np.random.randn(dimensions[i], dimensions[i + 1]) / np.sqrt(dimensions[i])
            self.vel[i + 1] = np.random.randn(dimensions[i], dimensions[i + 1]) / np.sqrt(dimensions[i])  
            self.b[i + 1] = np.zeros(dimensions[i + 1])
            self.activations[i + 2] = activations[i]
            
    def compute_dropout(self, activations, dropout_prob):
        """Sets a proportion of the activations to zero
        """
            
        activations/=dropout_prob    
        mult = np.random.binomial(1, 1- dropout_prob, size = activations.shape)
        activations*=mult
        return activations
    
    def _feed_forward(self, x, do_dropout, dropout_prob):
        """
        Execute a forward feed through the network.
        :param x: (array) Batch of input data vectors.
        :return: (tpl) Node outputs and activations per layer.
        """
        # w(x) + b
        z = {}

        # activations: f(z)
        a = {1: x}  # First layer has no activations as input. The input x is the input.

        for i in range(1, self.n_layers):
            z[i + 1] = np.dot(a[i], self.w[i]) + self.b[i]
            if i < (self.n_layers - 1):
                if do_dropout:
                    a[i + 1] = self.compute_dropout(self.activations[i + 1].activation(z[i + 1]), self.dropout_prob)
                else:
                    a[i + 1] = self.activations[i + 1].activation(z[i + 1])
            if i == (self.n_layers - 1):
         
                a[i + 1] = z[i + 1]

        return z, a

    def _back_prop(self, z, a, y_true, momentum, l2):
        """
        The input dicts keys represent the layers of the net.
        a = { 1: x,
              2: f(w1(x) + b1)
              3: f(w2(a2) + b2)}
        :param z: (dict) w(x) + b
        :param a: (dict) f(z)
        :param y_true: (array) One hot encoded truth vector.
        :return:
        """

        # Determine partial derivative and delta for the output layer.
        # delta output layer
        delta = self.loss.delta(y_true, a[self.n_layers])
        dw = np.dot(a[self.n_layers - 1].T, delta)
        
        update_params = {
            self.n_layers - 1: (dw, delta)
        }

        # In case of three layer net (two hidden layers) will iterate over i = 3 and i = 2
        # Determine partial derivative and delta for the rest of the layers.
        # Each iteration requires the delta from the previous layer, propagating backwards.
        for i in reversed(range(2, self.n_layers)):
            delta = np.dot(delta, self.w[i].T) * self.activations[i].prime(z[i])
            dw = np.dot(a[i - 1].T, delta) + (self.l2/len(y_true))*self.w[i-1]
            L2_term = ((self.l2/(2*y_true.shape[0])) * (np.sum(self.w[i-1] ** 2.) + np.sum(self.w[i] ** 2.)))
            delta += L2_term
            update_params[i - 1] = (dw, delta)
        
        for k, v in update_params.items():
            self._update_w_b(k, v[0], v[1], momentum)

    def _update_w_b(self, index, dw, delta, momentum):
        """
        Update weights and biases.
        :param index: (int) Number of the layer
        :param dw: (array) Partial derivatives
        :param delta: (array) Delta error.
        """

        self.w[index] -= self.learning_rate * dw
        self.b[index] -= self.learning_rate * np.mean(delta, 0)
        
        # momentum
        if momentum:  
            self.w[index], self.vel[index] = Momentum(self.w[index], self.vel[index], self.learning_rate, dw).update()
        else:
            self.w[index] -= self.learning_rate * dw
        

    def fit(self, x, y_true, loss, epochs, batch_size, learning_rate, learning_rate_decay, momentum, do_dropout, dropout_prob, l2):
        """
        :param x: (array) Containing parameters
        :param y_true: (array) Containing one hot encoded labels.
        :param loss: Loss class (MSE, CrossEntropy etc.)
        :param epochs: (int) Number of epochs/iterations.
        :param batch_size: (int) Number of samples in minibatch
        :param learning_rate: (flt)
        :param momentum
        """
        
        if not x.shape[0] == y_true.shape[0]:
            raise ValueError("Length of x and y arrays don't match")
        # Initiate the loss object with the final activation function
        self.l2 = l2
        self.loss = loss(self.activations[self.n_layers])
        self.learning_rate = learning_rate
        self.learning_rate_decay = learning_rate_decay
        self.do_dropout = do_dropout
        self.dropout_prob = dropout_prob
        loss_list = []
        acc_list = []
        for i in range(epochs):
            # Shuffle the data
            seed = np.arange(x.shape[0])
            np.random.shuffle(seed)
            x_ = x[seed]
            y_ = y_true[seed]
            # minibatch
            for j in range(x.shape[0] // batch_size):
                k = j * batch_size
                l = (j + 1) * batch_size
                z, a = self._feed_forward(x_[k:l], self.do_dropout, self.dropout_prob)
                self._back_prop(z, a, y_[k:l], momentum, l2)

            if (i + 1) % 10 == 0:
                _, a = self._feed_forward(x, self.do_dropout, self.dropout_prob)
                loss = self.loss.loss(y_true, a[self.n_layers])
                loss_list = np.append(loss_list, loss)
                print("Loss in %s epochs: %f" % (i + 1, loss))
                
                t_results = nn.predict(x_test,False, 0)
                correct = 0
                for i in range(len(y_test)):
                    pred = np.argmax(t_results[i])
                    true = y_test[i]
                    if pred == true :
                        correct += 1
                print('Accuracy = '+ str(correct/len(y_test)*100) \
                      + ' of ' + str(len(y_test)) \
                      + ' Samples. ')
                acc_list = np.append(acc_list,correct/len(y_test))
            # Learning rate decay each epochs to avoid local minima  
            self.learning_rate = self.learning_rate * (self.learning_rate / (self.learning_rate + (self.learning_rate * self.learning_rate_decay)))

        return i, loss_list, acc_list
    
    
    def predict(self, x, do_dropout, dropout_prob):
        self.do_dropout = do_dropout
        self.dropout_prob = dropout_prob
        """
        :param x: (array) Containing parameters
        :return: (array) A 2D array of shape (n_cases, n_classes).
        """
        _, a = self._feed_forward(x, self.do_dropout, self.dropout_prob)
        return a[self.n_layers]



In [4]:
#This Section runs the training model
if __name__ == "__main__":
    with h5py.File(training_data_file_location,'r') as H: data = np.copy(H['data'])
    with h5py.File(training_label_file_location,'r') as H: label = np.copy(H['label'])

    #Set Validation set
    start = 40000  #pic first record from data for validation
    end = 50000 ##pic last record from data for validation
    x_test = data[start:end]
    y_test = label[start:end]
 
    #Unused data normalisationo
    #data = (data - data.min()) / (data.max() - data.min())
    
    x = data[:40000] #Creation of training set (records have)
    y = label[:40000]

    y = np.eye(10)[y]

    
    nn = Network((128, 96, 54, 10), (Relu, Tanh, Softmax))
    e,losses,acc = nn.fit(x, y, loss=CrossEntropy, epochs=500, \
           batch_size=50, learning_rate=1e-4, learning_rate_decay=0.0001, \
           momentum = True, do_dropout = True, dropout_prob = 0.5, l2 =0.0)
  

<class '__main__.Softmax'>


C:\Users\Ryan\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:70: RuntimeWarning: invalid value encountered in log


Loss in 10 epochs: 1.067303
Accuracy = 73.49 of 10000 Samples. 
Loss in 20 epochs: 0.432268
Accuracy = 82.47 of 10000 Samples. 


In [5]:
#In Sample Testing

if analysis:
    prediction = nn.predict(x, do_dropout = False, dropout_prob = 0.5)

    y_true = []
    y_pred = []
    for i in range(len(y)):
        y_pred.append(np.argmax(prediction[i]))
        y_true.append(np.argmax(y[i]))

    print(metrics.classification_report(y_true, y_pred))


In [6]:
#Validation Testing without dropout
if analysis:
    prediction = nn.predict(x_test, do_dropout = False, dropout_prob = 0)
    #print(prediction)
    y_true = []
    y_pred = []
    for i in range(len(y_test)):
        y_pred.append(np.argmax(prediction[i]))
        y_true.append((y_test[i]))

    print(metrics.classification_report(y_true, y_pred))

In [7]:
#Run prediction over provided held out Testing Set for accuracy
if analysis:
    start_ = 50000  #pic first record from testing set 
    end_ = 60000 ##pic last record from testing set
    x_test_ho = data[start_:end_]
    y_test_ho = label[start_:end_]

    t_results = nn.predict(x_test_ho, do_dropout = False, dropout_prob = 0.)

    correct = 0
    for i in range(len(y_test_ho)):
        pred = np.argmax(t_results[i])
        true = y_test_ho[i]
        if pred == true :
            correct += 1
    print('Accuracy = '+ str(correct/len(y_test_ho)*100) + ' of ' + str(len(y_test_ho)) + ' Samples')

In [8]:
#Plot Results
if analysis:
    e = np.arange(1,len(losses)+1)
    pl.figure(figsize=(15,4))
    pl.plot(e*10, losses)
    pl.plot(e*10, acc)
    pl.grid()
    pl.grid(color='w', linestyle='-', linewidth=1)
    pl.xlabel('Epochs')
    pl.ylabel('Loss & Accuracy')


    pl.suptitle('Loss in training and Accuracy in Validation over Epochs', fontsize=16)


    pl.show()

In [9]:
#Testing of Real With held data for Marking
x_test_final = final_test_data
y_test_final = final_test_label

results = nn.predict(x_test_final, do_dropout = False, dropout_prob = 0.)

correct = 0
for i in range(len(y_test_final)):
    pred = np.argmax(results[i])
    true = y_test_final[i]
    if pred == true :
        correct += 1
print('Accuracy = '+ str(correct/len(y_test_final)*100) + ' of ' + str(len(y_test_final)) + ' Samples')

Accuracy = 82.515 of 60000 Samples


In [10]:
#Testing of Real With held data for Marking
prediction = nn.predict(x_test_final, do_dropout = False, dropout_prob = 0)

y_true = []
y_pred = []
for i in range(len(y_test_final)):
    y_pred.append(np.argmax(prediction[i]))
    y_true.append((y_test_final[i]))

print(metrics.classification_report(y_true, y_pred))
print('Final Model Accuracy = '+ str(correct/len(y_test_final)*100) + ' of ' + str(len(y_test_final)) + ' Samples')

             precision    recall  f1-score   support

          0       0.77      0.81      0.79      6000
          1       1.00      0.93      0.96      6000
          2       0.74      0.69      0.71      6000
          3       0.81      0.88      0.84      6000
          4       0.66      0.82      0.73      6000
          5       0.93      0.89      0.91      6000
          6       0.65      0.49      0.55      6000
          7       0.89      0.88      0.88      6000
          8       0.93      0.94      0.93      6000
          9       0.90      0.93      0.91      6000

avg / total       0.83      0.83      0.82     60000

Final Model Accuracy = 82.515 of 60000 Samples
